Assignment 3

# Build Question and Answer ChatBot from PDF document (Another approach: leverage Azure AI Search)

* Arxiv AI papers 


## Task 1: Use Azure OpenAI Studio load Arxiv AI papers




## Task 2: Set up Azure OpenAI



In [2]:
#Note: The openai-python library support for Azure OpenAI is in preview.

import os
import openai
from dotenv import load_dotenv
# Set up Azure OpenAI
load_dotenv()

openai.api_type = "azure"
openai.api_version = "2023-07-01-preview"

CHAT_API_KEY = os.getenv("OPENAI_API_CHAT_KEY","").strip()
assert CHAT_API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = CHAT_API_KEY

CHAT_RESOURCE_ENDPOINT = os.getenv("OPENAI_API_CHAT_ENDPOINT","").strip()
assert CHAT_RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in CHAT_RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = CHAT_RESOURCE_ENDPOINT

# Deployment for Chat
# DEPLOYMENT_NAME_CHAT = os.getenv('DEPLOYMENT_NAME_CHAT')
DEPLOYMENT_NAME_CHAT = os.getenv('DEPLOYMENT_NAME_CHAT_16K')

## Task 3: Set up Azure AI Search

In [4]:
import requests

deployment_id = DEPLOYMENT_NAME_CHAT

# Azure AI Search setup
search_endpoint = "https://yongaisearchservice.search.windows.net"; # Add your Azure AI Search endpoint here
# search_key = os.getenv("SEARCH_KEY"); # Add your Azure AI Search admin key here
search_key = "14jbI4GODPsnWwml22frAAV1tQyM0b1gWb0P9B0jahAzSeBXq7OQ"

search_index_name = "arxivpapers"; # Add your Azure AI Search index name here

def setup_byod(deployment_id: str) -> None:
    """Sets up the OpenAI Python SDK to use your own data for the chat endpoint.

    :param deployment_id: The deployment ID for the model to use with your own data.

    To remove this configuration, simply set openai.requestssession to None.
    """

    class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):

        def send(self, request, **kwargs):
            request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
            return super().send(request, **kwargs)

    session = requests.Session()

    # Mount a custom adapter which will use the extensions endpoint for any call using the given `deployment_id`
    session.mount(
        prefix=f"{openai.api_base}/openai/deployments/{deployment_id}",
        adapter=BringYourOwnDataAdapter()
    )

    openai.requestssession = session

setup_byod(deployment_id)

## Task 4: Build a Q&A ChatBot using own data

* Arxiv papers stored in Azure storage container

* Azure AI Search - vector DB

#### This the main loop for ChatBot conversation till user say 'quit'

* TODO: need to test more

In [9]:
import openai

# Chat history
message_history = [
    {
        "role": "system",
        "content": "You are an AI assistant that helps people find information."
    },
    {
        "role": "user",
        "content": "\"Use below context:...[your provided message context here]"
    }
]

# Function to handle bot responses
def chat_with_bot():
    while True:
        # 0. Get user input
        user_input = input("You: ")
        print("Me:", user_input)
        if user_input.lower() == 'quit':
            print("Exiting conversation.")
            break  # Exit the loop if user inputs 'quit'

        # message_text = [{"role": "user", "content": "What are the differences between Azure Machine Learning and Azure AI services?"}]

        # Append user input to message history
        # message_history.append({"role": "user", "content": user_input})

        message_text = [{"role": "user", "content": user_input}]

        completion = openai.ChatCompletion.create(
            messages=message_text,
            deployment_id=deployment_id,
            dataSources=[  # camelCase is intentional, as this is the format the API expects
                {
                    "type": "AzureCognitiveSearch",
                    "parameters": {
                        "endpoint": search_endpoint,
                        "key": search_key,
                        "indexName": search_index_name,
                    }
                }
            ],
            temperature=0.1,
            max_tokens=400,
        )
        # print(completion)
        
        # 5. Get bot response
        # bot_response = completion['choices'][0]['message']['content']
        bot_response = completion
        print("Bot:", bot_response)

        # 6. Append bot response to message history
        # message_history.append({"role": "assistant", "content": bot_response})
        # print("message_history:", message_history)

# Start the conversation
print("Start chatting with the bot ('quit' to exit):")
chat_with_bot()

Start chatting with the bot ('quit' to exit):
Me: "Model-Based Minimum Bayes Risk Decoding"
Bot: {
  "id": "0485851c-12e8-4214-bddb-d48020ce5a48",
  "model": "gpt-35-turbo-16k",
  "created": 1700154268,
  "object": "chat.completion",
  "choices": [
    {
      "index": 0,
      "messages": [
        {
          "index": 0,
          "role": "tool",
          "content": "{\"citations\": [{\"content\": \"Model-Based Minimum Bayes Risk Decoding\\nYuu Jinnai, Tetsuro Morimura, Ukyo Honda, Kaito Ariu, Kenshi Abe\\nCyberAgent\\n{jinnai_yu,morimura_tetsuro,honda_ukyo,kaito_ariu,abe_kenshi}@cyberagent.co.jp\\nAbstract\\nMinimum Bayes Risk (MBR) decoding has\\nbeen shown to be a powerful alternative to\\nbeam search decoding in a variety of text gener-\\nation tasks. MBR decoding selects a hypothesis\\nfrom a pool of hypotheses that has the least ex-\\npected risk under a probability model according\\nto a given utility function. Since it is impracti-\\ncal to compute the expected risk exactly 